In [ ]:
# Load AIEEE Data

In [1]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [2]:
!nvidia-smi

Fri Apr  1 09:27:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   31C    P0    34W / 250W |   2608MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"    
import tensorflow as tf

In [3]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [4]:
er_df = pd.read_csv("Models/Data/ERData/er_data_with_majority_label.csv")
er_df = er_df.dropna()

In [5]:
er_df.shape


(7149488, 3)

Load Models

In [6]:
#  other models
AIEEE_Muril = "Models/AIEEEGenderData/MurilBERT/SavedModel/MurilBERT"
CBSE_Muril = "Models/CBSEData/MurilBERT/SavedModel/MurilBERT"
ER_Muril = "Models/ERData/MurilBERT/SavedModel/BERTMuril"

In [7]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [8]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [9]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_Muril)

In [10]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_Muril)

In [11]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_Muril)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

Predict Gender

In [13]:
from transformers import pipeline
from sklearn.metrics import classification_report


In [19]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer, device=0)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
        with open("Models/ERData/er_data_cbse_test.txt", 'a') as f:
            f.write(f"{i}\n")
        pd.DataFrame
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    report = classification_report(true, labels, digits = 4)
    print(classification_report(true, labels, digits = 4))
    with open("Models/ERData/er_data_cbse_test.txt", 'a') as f:
            f.write(f"{report}\n")
    return report

In [18]:
report2 = print_results(cbse_model, tokenizer, er_df[:2000000])

              precision    recall  f1-score   support

         0.0     0.7843    0.6298    0.6986    999994
         1.0     0.6907    0.8268    0.7526   1000006

    accuracy                         0.7283   2000000
   macro avg     0.7375    0.7283    0.7256   2000000
weighted avg     0.7375    0.7283    0.7256   2000000



In [20]:
report3 = print_results(cbse_model, tokenizer, er_df[2000000:])

              precision    recall  f1-score   support

         0.0     0.7817    0.7901    0.7858   2336468
         1.0     0.8241    0.8167    0.8204   2813020

    accuracy                         0.8046   5149488
   macro avg     0.8029    0.8034    0.8031   5149488
weighted avg     0.8048    0.8046    0.8047   5149488



In [16]:
print(report2)

NameError: name 'report2' is not defined

In [ ]:
print(report1)

In [15]:
er_df = er_df.dropna()

In [16]:
er_df.shape

(7149488, 3)